# Attacks on Neural Networks in a Lightweight Speech Pseudonymization Pipeline

## Imports

In [13]:
import importlib
pipeline = importlib.import_module("ASR-2024-anonymization-module-learning")

import util
from neural_model import NeuralModel
from attacks.jingleback import Jingleback
from metrics import attack_success_rate, clean_accuracy_drop

import torch

from torchattacks.attacks.fgsm import FGSM
from torchattacks.attacks.pgd import PGD

## Preparation

In [14]:
TARGET_LABEL = 0

util.set_global_seed(3131)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

if str(device) == "cuda":
    print(torch.cuda.get_device_name())

cuda
NVIDIA GeForce GTX 1050


## Attacks
### Backdoor Attacks: JingleBack

In [15]:
jingleback_attack = Jingleback(source_label=None, target_label=TARGET_LABEL)
model = NeuralModel(attack=jingleback_attack, device=device)

AttributeError: 'ellipsis' object has no attribute 'to'

### Evasion Attacks: FGSM & PGD

In [12]:
fgsm = FGSM(model.neural_network, eps=8/255)
fgsm.set_mode_targeted_by_label() #NOTE: This means that, when attacking the model, you should pass the target label manually/yourself. So fgsm(audio, target_label).

NameError: name 'model' is not defined

In [ ]:
fgsm = PGD(model.neural_network, eps=8/255, alpha=2/255, steps=1, random_start=True)
fgsm.set_mode_targeted_by_label() #NOTE: This means that, when attacking the model, you should pass the target label manually/yourself. So fgsm(audio, target_label).